In [8]:
import subprocess as sp
import os
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

In [9]:
def listall(RootFolder, varname='',extension='.png'):
    lists = [os.path.join(root, name)
             for root, dirs, files in os.walk(RootFolder)
             for name in files
             if varname in name
             if name.endswith(extension)]
    return lists

# see note below
def youCanQuoteMe(item):
    return "\"" + item + "\""   

In [10]:
files = listall(r'G:\data\MODIS\MYD13A2005', varname='h27v05', extension='.hdf')

In [11]:
index = []
for i in files:
    # get date
    year = int(i[-36:-32])
    doy = int(i[-32:-29])
    date = datetime(year, 1, 1) + timedelta(doy - 1)
    date = np.datetime64(date)
    date = pd.Timestamp(np.datetime_as_string(date))
    index.append(date)
index = np.array(index)
df = pd.DataFrame(index=index)

In [12]:
month = df.index.map(lambda x: x.month)
year = df.index.map(lambda x: x.year)
day = df.index.map(lambda x: x.day)
doy =  df.index.map(lambda x: x.dayofyear)
# select only dates between certain month
df_sel = (df[(month >= 1) & (month <=6)] + 
          df[(month >= 7) & (month <=12)]).sort_index()
index_sel = pd.Series(np.in1d(df.index,df_sel.index))

In [13]:
len(index_sel), len(files)

(307, 307)

In [14]:
# prulletaria
gdal_translate = r'C:\Program Files\GDAL//gdal_translate.exe'
gdalwarp = r'C:\Program Files\GDAL//gdalwarp.exe'
out_base = r'D:\Data\NDAI_VHI_GROUNDTRUTH\tif\myd13a2//'

extent = [116.84693268,117.44,33.26,34.10324112]

for idx, val in enumerate(index_sel):    
    if val == True:
        print idx
        # 1. convert from HDF to TIF
        out_file = out_base+'MOD13A2_'+str(index[idx].year)+'-'+str(index[idx].month).zfill(2)+'-'+str(index[idx].day).zfill(2)
        src = 'HDF4_EOS:EOS_GRID:'+files[idx]+':MODIS_Grid_16DAY_1km_VI:1 km 16 days NDVI'        
        command = [gdal_translate,'-of','GTiff', src, out_file+'.tif']
        #print (sp.list2cmdline(command))
        norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
        norm.communicate()        
        
        # 2. reproject from sinusoidal to epsg 4326 (resampling also should take place in this step, if necessary)
        command = [gdalwarp, '-t_srs','EPSG:4326','-tr', '0.01', '0.01', '-overwrite', out_file+'.tif', out_file+'_rep.tif']
        #print (sp.list2cmdline(command))
        norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
        norm.communicate()                

        # 3. clip raster
        command = [gdal_translate, '-projwin', str(extent[0]),str(extent[3]),
                   str(extent[1]),str(extent[2]),'-of','GTiff', out_file+'_rep.tif', out_file+'_clip.tif']
        #print (sp.list2cmdline(command))
        norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
        norm.communicate()        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [18]:
# rename from iso date to date + dayofyear
files = listall(r'D:\Data\NDAI_VHI_GROUNDTRUTH\tif\mod13a2_myd13a2', extension='.tif')
for i in files:
    try:
        #print i
        year = int(i[-19:-15])
        month = int(i[-14:-12])
        day = int(i[-11:-9])
        date = datetime(year, month, day)
        date = np.datetime64(date)
        date = pd.Timestamp(np.datetime_as_string(date))    
        #print i[:-27] + 'MD13A2_'+str(date.year) +'_'+ str(date.dayofyear).zfill(3) +'.tif'
        os.rename(i, i[:-27] + 'MD13A2_' + str(date.year) +'_'+ str(date.dayofyear).zfill(3) +'.tif')
    except: 
        continue